# **⚡ SAM Algorithm**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## **🧮 Intermolecular Potential**

In [2]:
# Lennard-Jones potential between all pairs
def intermolecular_pair_potential(coordinates, epsilon=1.0, sigma=1.0):
    total_energy = 0.0
    n = len(coordinates)
    for i in range(n - 1):
        for j in range(i + 1, n):
            r = np.linalg.norm(coordinates[i] - coordinates[j])
            # Avoid division by zero or too small r
            if r == 0:
                continue
            total_energy += 4 * epsilon * ((sigma / r) ** 12 - (sigma / r) ** 6)
    return total_energy

## **🧬 Annealing Lennard-Jones**

In [3]:
def adaptive_simulated_annealing_lj(initial_coords, epsilon=1.0, sigma=1.0, 
                                    initial_step=0.05, initial_temp=10, max_iter_factor=2000):
    n_particles = initial_coords.shape[0]
    max_iterations = max_iter_factor * n_particles

    coords = initial_coords.copy()
    current_energy = intermolecular_pair_potential(coords, epsilon, sigma)
    best_coords = coords.copy()
    best_energy = current_energy

    T = initial_temp
    step_size = initial_step
    accept_count = 0
    adapt_interval = 1000  # how often to adapt step size and temperature

    for iteration in range(1, max_iterations + 1):
        # Propose new coords by adding Gaussian noise scaled by current step size
        proposal = coords + np.random.normal(scale=step_size, size=coords.shape)
        proposal_energy = intermolecular_pair_potential(proposal, epsilon, sigma)

        if proposal_energy < current_energy:
            # Accept only improvements
            coords = proposal
            current_energy = proposal_energy
            accept_count += 1

            # Update best if improved
            if current_energy < best_energy:
                best_energy = current_energy
                best_coords = coords.copy()

        # Adapt step size and temperature every adapt_interval iterations
        if iteration % adapt_interval == 0:
            acceptance_rate = accept_count / adapt_interval
            accept_count = 0

            # Adapt step size: if acceptance rate is too low, reduce step size; if too high, increase
            if acceptance_rate < 0.2:
                step_size *= 0.9
            elif acceptance_rate > 0.5:
                step_size *= 1.1

            # Decrease temperature adaptively
            T *= 0.9

            print(f"Iter {iteration:6d} | Temp {T:.5f} | Step size {step_size:.5f} | Best Energy {best_energy:.5f} | Acceptance Rate {acceptance_rate:.3f}")

        # Optional stopping criterion if step size becomes very small
        if step_size < 1e-5:
            print("Step size too small, stopping optimization.")
            break

    print(f"Best energy found: {best_energy:.5f}")
    return best_coords, best_energy

## **🧪 Testing**

In [17]:
# Example usage
initial_coords = np.random.rand(38, 3)  # 10 particles in 3D

optimized_coords, final_energy = adaptive_simulated_annealing_lj(initial_coords)

print("Optimized Coordinates:\n", optimized_coords)
print("Final Potential Energy:", final_energy)

Iter   1000 | Temp 9.00000 | Step size 0.04500 | Best Energy 511669621.35766 | Acceptance Rate 0.017
Iter   2000 | Temp 8.10000 | Step size 0.04050 | Best Energy 39359528.72915 | Acceptance Rate 0.011
Iter   3000 | Temp 7.29000 | Step size 0.03645 | Best Energy 35937465.41382 | Acceptance Rate 0.002
Iter   4000 | Temp 6.56100 | Step size 0.03281 | Best Energy 3726623.44093 | Acceptance Rate 0.016
Iter   5000 | Temp 5.90490 | Step size 0.02952 | Best Energy 766449.79496 | Acceptance Rate 0.019
Iter   6000 | Temp 5.31441 | Step size 0.02657 | Best Energy 174787.95611 | Acceptance Rate 0.025
Iter   7000 | Temp 4.78297 | Step size 0.02391 | Best Energy 6623.20538 | Acceptance Rate 0.059
Iter   8000 | Temp 4.30467 | Step size 0.02152 | Best Energy 41.61441 | Acceptance Rate 0.078
Iter   9000 | Temp 3.87420 | Step size 0.01937 | Best Energy -116.48547 | Acceptance Rate 0.048
Iter  10000 | Temp 3.48678 | Step size 0.01743 | Best Energy -131.27138 | Acceptance Rate 0.023
Iter  11000 | Temp 3.1